# Tutorial for field circuit coupling

<span style="color:red">Remark: To run this tutorial, the package `schemdraw` needs to be installed. It can be installed with:</span>
```shell
pip install schemdraw
```

In this tutorial, an axisymmetric pot inductor is simulated. A coil is wound around an iron core. For the coil there are two different conductor models. First, the single winding sare resolved and a [solid conductor model](#Solid-conductor-model) is used. After that, all wires are combined for the [stranded conductor model](#Stranded-condutctor-model).  
For both conductor models, the system is build, solved and some postprocessing is done.

The code is also in the file *tutorial.py* in the same folder as this tutorial.

Note that with the current version of field-circuit coupling, only the most simple circuits of a current or a voltage source are supported directly. Field-circuit coupling with more complex circuits (as it is done in the section [Coupling to an external circuit](#Coupling-to-an-external-circuit)) has to be done by hand at the time. 

<span style="color:red">Note that some cells fail when executed for the first time. Rerunning them is sufficient.</span>

In [ ]:
# General imports
import numpy as np
import scipy.sparse.linalg as splinalg
import gmsh
import matplotlib.pyplot as plt
from scipy import sparse

In [ ]:
# Imports from pyrit
from pyrit.excitation import FieldCircuitCoupling, StrandedConductor, SolidConductor
from pyrit.material import Reluctivity, Resistivity, Conductivity
from pyrit.geometry import Geometry

# Some functionality is moved to this file for better readibility
import tutorial

# Solid conductor model
We begin with the solid conductor model. As already said, all the single wires are resolved in this model. The wires can be seen in the plots. The problem is defined in the class *CoilInYokeSolid* from the *tutorial* module. 

## Preparation and solving
### Create the problem with solid conductors
At first, the problem is created and the frequency is set. In the ``create_geometry``-method, there are the (in this case 72) solid conductor models created. They are created with 

```python
>>> SolidConductor(current=1)
```

So the current is set to $1\mathrm{A}$ for each winding. 

In [ ]:
coil_in_yoke = tutorial.CoilInYokeSolid(windings=72, mu_r_iron=1000)
problem = coil_in_yoke.create_geometry(mesh_size=0.2, mesh_size_conductors=0.0005)

shape_function = problem.shape_function
mesh = problem.mesh

In [ ]:
problem.frequency = 1e2

### Build matrices
Then the matrices are build. The *mass_one* matrix is only needed for the postprocessing.

In [ ]:
curlcurl = shape_function.curlcurl_operator(problem.regions, problem.materials, Reluctivity)
mass = shape_function.mass_matrix(problem.regions, problem.materials, Conductivity)
mass_one = shape_function.mass_matrix(1)
rhs = shape_function.load_vector(0)
matrix = curlcurl + 1j * problem.omega * mass

### Shrink, solve and inflate
The system of equation is getting shrunk, then solved and inflated. 

<span style="color:red">Note that here the shrink and inflate methods from *FieldCircuitCoupling* are used. This is an important difference to simulations without field circuit coupling.</span>

In [ ]:
matrix_shrink, rhs_shrink = FieldCircuitCoupling.shrink(matrix, rhs, problem, integration_order=2)
solution_shrink = splinalg.spsolve(matrix_shrink.tocsr(), rhs_shrink.tocsr())
solution, ids = FieldCircuitCoupling.inflate(solution_shrink, problem)

## Postprocessing
At first, we determine the voltage drop over all wires by adding the single voltage drops over all wires. Together with the current of 1A, the impedance can be determined.

In [ ]:
# Prepare some data
nodes_not_on_axis = np.setdiff1d(np.arange(mesh.num_node), mesh.nodes_on_axis)
voltage = sum((problem.excitations.get_exci(id).voltage for id in ids))
print(f"Complex voltage: {voltage}")
impedance = voltage / 1
resistance = impedance.real
inductance = impedance.imag / problem.omega

print(f"Resistance: {resistance}\nInductance: {inductance}")

### Current and Voltage over time
Here we plot the current and voltage over time (acutally over 3 periods). For that we use the definition of a phasor.

In [ ]:
# %matplotlib notebook
time = np.linspace(0,3/problem.frequency, 200)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(time,np.cos(problem.omega*time), label='Current')
ax.plot(time,np.abs(voltage)*np.cos(problem.omega*time + np.angle(voltage)), label="voltage")
ax.legend()

### Magnetic flux density
Here we plot the magnetic flux density. First, the complex vecvtor **b** is calculated. For plotting, we take the real part of **b** (that means one specific time instance) and then the absulute value of the vector. So in the end we plot the absolute value of the magnetic flux density of one specific time instance.

In [ ]:
# %matplotlib notebook
b = shape_function.curl(solution)
mesh.plot_scalar_field(np.hypot(np.real(b[:, 0]), np.real(b[:, 1])), title="Magnetic flux density")

### Plotting electic field and current density
The electric field consists of two parts. One part from the eddy currents and one from the field circuit coupling (here denoted as *source*). The former is determined with $$\vec{E}_\mathrm{eddy} = -\jmath\omega\vec{A}$$ and the latter is written with the voltage of every wire and the corresponding voltage distribution function: $$\vec{E}_\mathrm{source} = \sum_k V_k\vec{\zeta}_k$$. 
The useage of the methods *project_to_c_vector* and *field_vector* are due to the fact that here edge functions in an axisymmetric coordinate system are used. 

In [ ]:
# %matplotlib notebook
e_eddy = -1j * problem.omega * solution

e_source = sparse.coo_matrix((mesh.num_node,1))
for key in ids:
    fcc: SolidConductor = problem.excitations.get_exci(key)
    fcc.compute_voltage_distribution_function(problem)
    e_source = e_source + fcc.voltage * fcc.voltage_distribution_function

e = e_eddy+e_source.toarray().flatten()
j = tutorial.project_to_c_vector(mass @ e, mass_one.tocsr(), mesh.num_node, nodes_not_on_axis)
e_field = tutorial.field_vector(e, mesh, nodes_not_on_axis)
j_field = tutorial.field_vector(j, mesh, nodes_not_on_axis)
mesh.plot_scalar_field(np.real(e_field), title="Electric field")
mesh.plot_scalar_field(np.real(j_field), title="Current density")

# Stranded condutctor model
Now we use the stranded conductor model for the coil. The single strands are no longer resolved by the mesh. Instead of a current flowing in each strand, there is a current flowing on the whole coil domain.

## Preparing and solving

### Create the problem
The problem is created and the frequency set. This functionality is in *tutorial*.

In [ ]:
coil_in_yoke = tutorial.CoilInYokeStranded(mu_r_iron=1000, fill_factor=0.565486)
sc = StrandedConductor(voltage=1e-3, windings=72)
problem = coil_in_yoke.create_geometry_stranded(sc)

shape_function = problem.shape_function
mesh = problem.mesh

In [ ]:
problem.frequency = 5e2

### Build matrices
The needed matrices are build. The *mass_one* and *mass_rho* matrix are only for the postprocessing needed.

In [ ]:
curlcurl = shape_function.curlcurl_operator(problem.regions, problem.materials, Reluctivity)
mass = shape_function.mass_matrix(problem.regions, problem.materials, Conductivity)
mass_one = shape_function.mass_matrix(1)
mass_rho = shape_function.mass_matrix(problem.regions, problem.materials, Resistivity)
rhs = shape_function.load_vector(0)
matrix = curlcurl + 1j*problem.omega * mass

### Shrink, solve, inflate
The system of equation is getting shrunk, then solved and inflated. 

<span style="color:red">Note that here the shrink and inflate methods from *FieldCircuitCoupling* are used. This is an important difference to simulations without field circuit coupling.</span>

In [ ]:
matrix_shrink, rhs_shrink = FieldCircuitCoupling.shrink(matrix, rhs, problem, integration_order=2)
solution_shrink = splinalg.spsolve(matrix_shrink.tocsr(), rhs_shrink.tocsr())
solution, fcc_ids = FieldCircuitCoupling.inflate(solution_shrink, problem)

## Postprocessing
At first we determine the current through the coil. Note that this is the curent through one strand. With that we can determine the resistance and the inductance of the coil.

In [ ]:
nodes_not_on_axis = np.setdiff1d(np.arange(mesh.num_node), mesh.nodes_on_axis)

current = sc.current
voltage = sc.voltage
impedance = sc.impedance
resistance = sc.resistance
inductance = sc.inductance(problem.omega)

print(f"Resistance: {resistance}\nInductance: {inductance}")

### Voltage and current over time

In [ ]:
time = np.linspace(0, 3 / problem.frequency, 200)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(time, sc.voltage*np.cos(problem.omega * time), label='Voltage')
ax.plot(time, np.abs(current) * np.cos(problem.omega * time + np.angle(current)), label="current")
ax.legend()

### Flux density

In [ ]:
b = shape_function.curl(solution)
mesh.plot_scalar_field(np.hypot(np.real(b[:, 0]), np.real(b[:, 1])), title="flux density")

### Electric field and current density

In [ ]:
e_eddy = -1j * problem.omega * solution
j_source = None
for fcc in StrandedConductor.iter_fcc(problem.excitations):
    fcc.compute_current_distribution_function(problem)
    if j_source is None:
        j_source = fcc.current * fcc.windings * fcc.current_distribution_function
    else:
        j_source = j_source + fcc.current * fcc.windings * fcc.current_distribution_function

e_source = tutorial.project_to_c_vector(mass_rho @ j_source, mass_one.tocsr(), mesh.num_node, nodes_not_on_axis)

e = e_eddy + e_source
j = j_source.toarray().flatten() + tutorial.project_to_c_vector(mass @ e, mass_one.tocsr(), mesh.num_node, nodes_not_on_axis)
e_field = tutorial.field_vector(e, mesh, nodes_not_on_axis)
j_field = tutorial.field_vector(j, mesh, nodes_not_on_axis)

mesh.plot_scalar_field(np.real(e_field), title="e field")
mesh.plot_scalar_field(np.real(j_field), title="j field")

# Coupling to an external circuit
Now we want to show with an example how to couple the field model to an actual circuit. For that, we consider an easy circuit conststing of a resistance in series connection to a parallel connection of a capacitor and an inductance. For the inductance, we use the field model to determine the relation between voltage and current. The circuit is plottet in the next cell.

In [ ]:
tutorial.draw_circuit()

We first define some values:

In [ ]:
resistance = 2  # in ohm
capacitance = 1e-6  # in farad

Then we create the problem and get the matrices:

In [ ]:
windings = 72
coil_in_yoke = tutorial.CoilInYokeSolid(windings=windings, mu_r_iron=1000)
problem = coil_in_yoke.create_geometry(mesh_size=0.2, mesh_size_conductors=0.005)

problem.frequency = 1e4
shape_function = problem.shape_function
mesh = problem.mesh

curlcurl = shape_function.curlcurl_operator(problem.regions, problem.materials, Reluctivity)
mass = shape_function.mass_matrix(problem.regions, problem.materials, Conductivity)
mass_one = shape_function.mass_matrix(1)
rhs = shape_function.load_vector(0)
matrix = curlcurl + 1j * problem.omega * mass

Having the finite element matrices, we need to also build the circuit matrices and couple them. This is done in the next cell. 

In [ ]:
# Shrink the system
solid_conductors = SolidConductor.iter_fcc(problem.excitations)
bottom_matrices, right_matrices, diagonal_matrices, rhs_matrices = [], [], [], []

for fcc in solid_conductors:
    # get the coupling values
    # noinspection PyTupleAssignmentBalance
    fcc.compute_coupling_values(problem)

    bottom_matrices.append(fcc.bottom_coupling_matrix)
    right_matrices.append(fcc.right_coupling_matrix)
    diagonal_matrices.append(sparse.coo_matrix(np.array([[fcc.coupling_value]])))

# Assembly of the matrices
bottom_matrix: sparse.csr_matrix = sparse.vstack(bottom_matrices, format='csr')
right_matrix: sparse.csr_matrix = sparse.hstack(right_matrices, format='csr')
diagonal_matrix = sparse.block_diag(diagonal_matrices, format='csr')
rhs_fcc = sparse.coo_matrix((windings, 1))

# shrink the system with the sft
ret = problem.shape_function.shrink(matrix, rhs, problem, integration_order=2)
matrix_shrink_sf, rhs_shrink_sf, indices_not_dof, _, _ = ret

# compose the coupling matrices and coupling values with the information from the sft.shrink
all_indices = np.arange(rhs.shape[0])
# num_fcc = len(bottom_matrices)
new_lines = bottom_matrix.shape[0]
new_shape = matrix_shrink_sf.shape[0]
indices_dof = np.setdiff1d(all_indices, indices_not_dof)
bottom_matrix = bottom_matrix[np.ix_(np.arange(new_lines), indices_dof)]
right_matrix = right_matrix[np.ix_(indices_dof, np.arange(new_lines))]
bottom_matrix.resize((bottom_matrix.shape[0], new_shape))
right_matrix.resize((new_shape, right_matrix.shape[1]))

# compute the final system
matrix_shrink = sparse.bmat([[matrix_shrink_sf, right_matrix],
                             [bottom_matrix, diagonal_matrix]])
rhs_shrink = sparse.vstack([rhs_shrink_sf, rhs_fcc])

At this point we have to change the matrix and the right-hand side. The following equations also have to be solved: $$ I_s = I_c + I_L\quad\Leftrightarrow I_L + \jmath\omega CV - \frac{V_R}{R} = 0$$ $$V + V_R = 0\,.$$
Furthermore, the existing matrix and right-hand side have to be adapted in oder to have the voltage drop over the coil $V$ and the current through the coil $I_L$ in the solution vector. Denoting $V_k$ as the voltage drop over the $k$-th strand, the bottom right part of the final system has the following form: 
$$\begin{pmatrix}
			\ast &        &      & -1     & 0 & 0 \\
			     & \ddots &      & \vdots & 0 & 0 \\
			     &        & \ast & -1     & 0 & 0 \\
			-1   & \dots & -1   &  0      & 1 & 0 \\
			0    & 0      & 0    & 1       & \jmath\omega C & -\frac{1}{R} \\
			0    & 0      & 0    & 0       & 1 & 1
		\end{pmatrix}\begin{pmatrix}V_1\\\vdots\\ V_k\\ I_L\\ V\\ V_R\end{pmatrix} = \begin{pmatrix}0\\0\\0\\0\\0\\ V_s\end{pmatrix}$$

In [ ]:
old_size = matrix_shrink.shape[0]
bottom_matrix = sparse.coo_matrix((-1*np.ones(windings),(np.zeros(windings),np.arange(old_size-windings,old_size))),shape=(3,old_size))
right_matrix = bottom_matrix.transpose()
bottom_right_matrix = np.array([[0,1,0],
                                [1, 1j*problem.omega*capacitance, -1/resistance],
                                [0,1,1]])
bottom_right_matrix = sparse.coo_matrix(bottom_right_matrix)

new_matrix = sparse.hstack([sparse.vstack([matrix_shrink,bottom_matrix]),sparse.vstack([right_matrix,bottom_right_matrix])],format='csr')

new_rhs = rhs_shrink.tocsr()
new_rhs = sparse.vstack([new_rhs,sparse.coo_matrix(([1],([2],[0])))],format='csr')

Now, the system can be solved, the solution extracted and the postprocessed data be plottet:

In [ ]:
solution = splinalg.spsolve(new_matrix,new_rhs)

I_L = solution[-3]
V = solution[-2]
V_R = solution[-1]

I_s = V_R/resistance
I_c = I_s - I_L

# solution, solution_values = FieldCircuitCoupling.inflate(solution[:-3], problem)

impedance = V/I_L
resistance = impedance.real
inductance = impedance.imag / problem.omega
print(f"Resistance: {resistance}\tInductance: {inductance}")

print(f"Voltages:\nTotal voltage: {1}\nVoltage over R: {V_R}\nVoltage over L,C: {V}\n")
print(f"Currents:\nTotal current: {I_s}\nCurrent through L: {I_L}\nCurrent through C: {I_c}\n")

In [ ]:
time = np.linspace(0, 3 / problem.frequency, 200)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(time, np.cos(problem.omega * time), label=r'$V_s$')
ax.plot(time, np.abs(V_R) * np.cos(problem.omega * time + np.angle(V_R)), label=r"$V_R$")
ax.plot(time, np.abs(V) * np.cos(problem.omega * time + np.angle(V)), label=r"$V$")
ax.set_title('Voltages')
ax.legend()

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(time, np.abs(I_s) * np.cos(problem.omega * time + np.angle(I_s)), label=r"$I_s$")
ax.plot(time, np.abs(I_c) * np.cos(problem.omega * time + np.angle(I_c)), label=r"$I_c$")
ax.plot(time, np.abs(I_L) * np.cos(problem.omega * time + np.angle(I_L)), label=r"$I_L$")
ax.set_title('Currents')
ax.legend()